In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
import random
random.sample([1,2,3], 1)[0]

3

In [4]:
from dataset import COCODataset, COCODataLoader

In [5]:
# 1. check dataloader
ds = COCODataset(split="valid", max_samples=1000)
dl = COCODataLoader(ds, batch_size={384: 5, 448:4, 512: 4}, num_workers=4)

Found annotations at '/home/lukovdg1/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 1000/1000 [10.7s elapsed, 0s remaining, 106.2 samples/s]      
Dataset 'coco-2017-validation-1000' created


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 110.63it/s]

[72, 83, 18]


In [10]:
for batch in iter(dl):
    print(batch["image"].shape)

torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5, 3, 384, 384])
torch.Size([5,

In [ ]:
class MyCLIPTextEmbedder(FrozenCLIPEmbedder):
    """Uses the CLIP transformer encoder for text (from huggingface)"""

    def forward(self, fullprompt):
        fullprompt, batsize = fullprompt if isinstance(fullprompt, tuple) else (fullprompt, 1)
        
        layer_idses = []
        text_embeddingses = []
        token_idses = []
        
        layertexts = fullprompt.split("\n|")
        
        # encode layers separately
        for i, pos_prompt in enumerate(layertexts):
            if i == 0:
                token_ids, layer_ids = _tokenize_annotated_prompt(pos_prompt.strip(), self.tokenizer)
                global_len = token_ids.shape[0]
                global_bos_pos = 0
                global_eos_pos = torch.nonzero(token_ids == self.tokenizer.eos_token_id)[0][0]
            else:
                pos_prompt = pos_prompt.strip()
                token_ids = self.tokenizer([pos_prompt], return_tensors="pt",
                                        max_length=self.max_length, return_overflowing_tokens=False,
                                        truncation=True)["input_ids"][0]
                layer_ids = torch.tensor([i] * token_ids.shape[0])
            outputs = self.transformer(input_ids=token_ids[None].to(self.device), output_hidden_states=self.layer=="hidden")
            if self.layer == "last":
                z = outputs.last_hidden_state
            elif self.layer == "pooled":
                z = outputs.pooler_output[:, None, :]
            else:
                z = outputs.hidden_states[self.layer_idx]
                
            layer_idses.append(layer_ids)
            token_idses.append(token_ids)
            text_embeddingses.append(z)
            
        layer_ids = torch.cat(layer_idses, 0)[None].repeat(batsize, 1)
        token_ids = torch.cat(token_idses, 0)[None].repeat(batsize, 1)
        text_embeddings = torch.cat(text_embeddingses, 1).repeat(batsize, 1, 1)
        global_prompt_mask = torch.zeros_like(token_ids)
        global_bos_eos_mask = torch.zeros_like(global_prompt_mask)
        global_prompt_mask[:, :global_len] = 1
        global_bos_eos_mask[:, global_bos_pos] = 1
        global_bos_eos_mask[:, global_eos_pos] = 1
        
        ret = CustomTextConditioning(text_embeddings, layer_ids.to(self.device), token_ids.to(self.device),
                                     global_prompt_mask.to(self.device), global_bos_eos_mask.to(self.device))
        return ret
        # return text_embeddings, layer_ids.to(self.device), token_ids.to(self.device)


In [ ]:
# 2. create model
model_name = 'control_v11p_sd15_seg'
model = create_model(f'./models/{model_name}.yaml').cpu()
# load main weights
model.load_state_dict(load_state_dict('./models/v1-5-pruned.ckpt', location='cuda'), strict=False)
# load controlnet weights
model.load_state_dict(load_state_dict(f'./models/{model_name}.pth', location='cuda'), strict=False)
model = model.cuda()

# cast text encoder to our own
model.cond_stage_model.__class__ = MyCLIPTextEmbedder

for _module in model.model.diffusion_model.modules():
    if _module.__class__.__name__ == "CrossAttention":
        _module.__class__.forward = custom_forward
